In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "tests/test_custom/config_app/"

In [2]:
import sys
import os
import pickle
os.chdir("..")
from omegaconf import OmegaConf

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine


/env/ds/anaconda/envs/pipeline/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from etl_pipeline.config import columns_namespace as cn

In [4]:
# alert = load_alert()

In [5]:
# payload = load_alert()


# Definition

In [6]:
from copy import deepcopy
from etl_pipeline.config import alert_agents_config, pipeline_config
from etl_pipeline.config import columns_namespace as cn
from etl_pipeline.custom.ms.datatypes.field import InputRecordField
from etl_pipeline.custom.ms.payload_loader import PayloadLoader
from etl_pipeline.custom.ms.transformations import (
    create_agent_input_agg_col_config,
    prepend_agent_name_to_ap_or_wl_or_aliases_key,
)
from etl_pipeline.custom.ms.watchlist_extractor import WatchlistExtractor
from etl_pipeline.pipeline import ETLPipeline


class MSPipeline(ETLPipeline):
    def convert_raw_to_standardized(self, df):
        return df
    
    def connect_input_record_with_match_record(self, payload):
        import pdb; pdb.set_trace()
        for input_record in payload[cn.ALERT_FIELD][cn.INPUT_RECORD_HIST]:
            input_record['INPUT_FIELD'] = {
                i["name"]: InputRecordField(**i)
                for i in input_record['field']
            }
            
        new_payloads = []
        for input_record in payload[cn.ALERT_FIELD][cn.INPUT_RECORD_HIST]:
            for match_record in payload[cn.ALERT_FIELD][cn.MATCH_RECORDS]:
                if input_record['versionId'] ==  match_record['inputVersionId']:
                    pair_payload = deepcopy(payload)
                    for num, input_record_to_remove in enumerate(payload[cn.ALERT_FIELD][cn.INPUT_RECORD_HIST]):
                        if input_record['versionId'] !=  input_record_to_remove['versionId']:
                            del pair_payload[cn.ALERT_FIELD][cn.INPUT_RECORD_HIST][num]
                                
                    for num, match_record_to_remove in  enumerate(payload[cn.ALERT_FIELD][cn.MATCH_RECORDS]):
                        if match_record['inputVersionId'] !=  match_record_to_remove['inputVersionId']:
                            del pair_payload[cn.ALERT_FIELD][cn.MATCH_RECORDS][num]
                            
                    new_payloads.append(pair_payload)
                    
        return new_payloads        
        
    def transform_standardized_to_cleansed(self, payload):
        match_ids = payload[cn.MATCH_IDS]
        matches = payload[cn.ALERT_FIELD][cn.MATCH_RECORDS]

        parties = payload[cn.SUPPLEMENTAL_INFO][cn.RELATED_PARTIES][cn.PARTIES]

        for num, party in enumerate(parties):
            parties[num] = party["fields"]
                   
        new_payloads = self.connect_input_record_with_match_record(payload)
        
        for payload in new_payloads:
            fields = payload[cn.ALERT_FIELD][cn.INPUT_RECORD_HIST][0]["INPUT_FIELD"]
            for match_id in match_ids:
                match = matches[match_id]
                WatchlistExtractor().update_match_with_wl_values(match)
                match[cn.TRIGGERED_BY] = self.engine.set_trigger_reasons(
                    match, self.pipeline_config.FUZZINESS_LEVEL
                )
                self.engine.set_beneficiary_hits(match)

            self.engine.connect_full_names(parties)

            self.engine.collect_party_values(parties, payload)
            payload[cn.ALL_CONNECTED_PARTY_TYPES] = payload[cn.ALL_PARTY_TYPES]
            names_source_cols = [
                cn.ALL_PARTY_NAMES,
                cn.ALL_CONNECTED_PARTIES_NAMES,
            ]

            payload.update(
                {
                    cn.CLEANED_NAMES: self.engine.get_clean_names_from_concat_name(
                        fields.get(cn.CONCAT_ADDRESS, None).value,
                        {key: payload[key] for key in names_source_cols},
                    )
                }
            )

            payload.update({cn.CONCAT_RESIDUE: payload[cn.CLEANED_NAMES][cn.CONCAT_RESIDUE]})

            concat_residue = payload[cn.CONCAT_RESIDUE]
            concat_address = fields.get(cn.CONCAT_ADDRESS, None).value

            payload.update({cn.CONCAT_ADDRESS_NO_CHANGES: concat_residue == concat_address})
            for match_id in match_ids:
                match = matches[match_id]
                match[cn.AP_TRIGGERS] = self.engine.set_triggered_tokens_discovery(
                    payload, match, fields
                )

        return new_payloads

    def get_key(self, payload, match, conf):
        new_config = {}
        for key, value in dict(conf).items():
            temp_dict = dict(value)
            for new_key in temp_dict:
                for element in temp_dict[new_key]:
                    elements = element.split(".")
                    if cn.MATCH_RECORDS in element:
                        value = match
                        elements = elements[1:]
                    else:
                        value = payload

                    for field_name in elements:
                        if field_name == "INPUT_FIELD":
                            value = value[0][field_name][elements[-1]].value
                            break
                        try:
                            value = value.get(field_name, None)
                        except TypeError:
                            key = PayloadLoader.LIST_ELEMENT_REGEX.sub("", field_name)
                            ix = int(PayloadLoader.LIST_ELEMENT_REGEX.match(field_name).groups(0))
                            value = value[key][ix]
                    new_config[elements[-1]] = value
        return new_config

    def load_agent_config(self, alert_type="WM_ADDRESS"):
        alert_config = alert_agents_config[alert_type]
        parsed_agent_config = {}
        for agent_name, agent_config in dict(alert_config).items():
            particular_agent_config = dict(agent_config)
            parsed_agent_config[agent_name] = {}
            for new_key in particular_agent_config:
                parsed_agent_config[agent_name][new_key] = []
                for element in particular_agent_config[new_key]:
                    elements = element.split(".")
                    parsed_agent_config[agent_name][new_key].append(elements[-1])
        return parsed_agent_config, alert_config

    def transform_cleansed_to_application(self, payload):
        new_payloads = payload
        for payload in new_payloads:
            match_ids = payload[cn.MATCH_IDS]
            matches = payload[cn.ALERT_FIELD][cn.MATCH_RECORDS]
            agent_config, yaml_conf = self.load_agent_config()
            agent_input_prepended_agent_name_config = prepend_agent_name_to_ap_or_wl_or_aliases_key(
                agent_config
            )
            
            agent_input_agg_col_config = create_agent_input_agg_col_config(
                agent_input_prepended_agent_name_config
            )

            for match_id in match_ids:
                match = matches[match_id]
                config = self.get_key(payload, match, yaml_conf)
                self.engine.sql_to_merge_specific_columns_to_standardized(
                    agent_input_prepended_agent_name_config,
                    match,
                    config,
                    False,
                )
                config.update(
                    {
                        key: match.get(key)
                        for key in match
                        if key.endswith("_ap") or key.endswith("_wl")
                    }
                )
                self.engine.sql_to_merge_specific_columns_to_standardized(
                    agent_input_agg_col_config, match, config, True
                )

        return new_payloads


In [7]:
with open(f'notebooks/sample/alert_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json['match_ids'] = [i for i in range(len(payload_json[cn.ALERT_FIELD][cn.MATCH_RECORDS]))]


{'createdDate': '01/08/20'}
{'field': [{'isScreenable': 'false'}]}
{'isScreenable': 'false'}
{'field': [{'name': 'SOURCE_REF'}]}
{'name': 'SOURCE_REF'}
{'field': [{'sortOrder': '1'}]}
{'sortOrder': '1'}
{'field': [{'value': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'}]}
{'value': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'}
{'field': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'isScreenable': 'false'}]}
{'isScreenable': 'false'}
{'field': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'name': 'ADDRESS1_LINE5'}]}
{'name': 'ADDRESS1_LINE5'}
{'field': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'sortOrder': '11'}]}
{'sortOrder': '11'}
{'field': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'isScreenable': 'false'}]}
{'isScreenable': 'false'}
{'field': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'name': 'ADDRESS1_LINE6'}]}
{'name': 'ADDRESS1_LINE6'}
{'field': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'sortOrder': '12'}]}
{'sortOrder': '12'}
{'field': [{}, {

In [8]:
payload

{'alert.headerInfo.uniqueCustomerId': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480',
 'alert.headerInfo.datasetId': '1044',
 'alert.headerInfo.datasetName': 'R_US_Active_Address',
 'alert.headerInfo.masterId': '72951854',
 'alert.headerInfo.currentVersionId': '122438658',
 'alert.headerInfo.stopDescriptors.stopDescriptor.name': 'Joe Doe',
 'alert.headerInfo.firstVersionCreatedDt': '2020-01-08T09:31:30.264-05:00',
 'alert.headerInfo.lastVersionUpdatedDt': '2020-01-08T09:31:30.264-05:00',
 'alert.headerInfo.masterVersion': '412740c151535f0756e8dbec7440b726c7a3e135',
 'alert.headerInfo.inputVersionSample': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.',
 'alert.inputRecordHist[0].version': '412740c151535f0756e8dbec7440b726c7a3e135',
 'alert.inputRecordHist[0].createdDate': '01/08/20',
 'alert.inputRecordHist[0].field[0].name': 'SOURCE_REF',
 'alert.inputRecordHist[0].field[0].isScreenable': 'false',
 'alert.inputRecordHist[0].field[0].value': 'R_US_Active_Address_A05

In [9]:
payload = payload_json

In [10]:
engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

## transform standardized to cleansed

In [11]:
payload

{'alert': {'headerInfo': {'currentVersionId': '122438658',
   'datasetId': '1044',
   'datasetName': 'R_US_Active_Address',
   'firstVersionCreatedDt': '2020-01-08T09:31:30.264-05:00',
   'inputVersionSample': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.',
   'lastVersionUpdatedDt': '2020-01-08T09:31:30.264-05:00',
   'masterId': '72951854',
   'masterVersion': '412740c151535f0756e8dbec7440b726c7a3e135',
   'stopDescriptors': {'stopDescriptor': {'name': 'Joe Doe'}},
   'uniqueCustomerId': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'},
  'inputRecordHist': [{'createdDate': '01/08/20',
    'field': [{'isScreenable': 'false',
      'name': 'SOURCE_REF',
      'sortOrder': '1',
      'value': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'},
     {'isScreenable': 'false',
      'name': 'UNIQUE_KEY',
      'sortOrder': '2',
      'value': 'A05003324172'},
     {'isScreenable': 'false', 'name': 'BARCODE', 'sortOrder': '3'},
     {'isScreenable': 'false',

In [12]:
self = pipeline
match_ids = payload[cn.MATCH_IDS]
parties = payload[cn.SUPPLEMENTAL_INFO][cn.RELATED_PARTIES][cn.PARTIES]

for num, party in enumerate(parties):
    parties[num] = party["fields"]

new_payloads = self.connect_input_record_with_match_record(payload)

for payload in new_payloads:
    matches = payload[cn.ALERT_FIELD][cn.MATCH_RECORDS]
    fields = payload[cn.ALERT_FIELD][cn.INPUT_RECORD_HIST][0]["INPUT_FIELD"]
    for match in matches:
        WatchlistExtractor().update_match_with_wl_values(match)
        match[cn.TRIGGERED_BY] = self.engine.set_trigger_reasons(
            match, self.pipeline_config.FUZZINESS_LEVEL
        )
        self.engine.set_beneficiary_hits(match)

    self.engine.connect_full_names(parties)

    self.engine.collect_party_values(parties, payload)
    payload[cn.ALL_CONNECTED_PARTY_TYPES] = payload[cn.ALL_PARTY_TYPES]
    names_source_cols = [
        cn.ALL_PARTY_NAMES,
        cn.ALL_CONNECTED_PARTIES_NAMES,
    ]

    payload.update(
        {
            cn.CLEANED_NAMES: self.engine.get_clean_names_from_concat_name(
                fields.get(cn.CONCAT_ADDRESS, None).value,
                {key: payload[key] for key in names_source_cols},
            )
        }
    )

    payload.update({cn.CONCAT_RESIDUE: payload[cn.CLEANED_NAMES][cn.CONCAT_RESIDUE]})

    concat_residue = payload[cn.CONCAT_RESIDUE]
    concat_address = fields.get(cn.CONCAT_ADDRESS, None).value

    payload.update({cn.CONCAT_ADDRESS_NO_CHANGES: concat_residue == concat_address})
    for match in matches:
        match[cn.AP_TRIGGERS] = self.engine.set_triggered_tokens_discovery(
            payload, match, fields
        )

> /tmp/ipykernel_370706/3292963255.py(20)connect_input_record_with_match_record()
     18     def connect_input_record_with_match_record(self, payload):
     19         import pdb; pdb.set_trace()
---> 20         for input_record in payload[cn.ALERT_FIELD][cn.INPUT_RECORD_HIST]:
     21             input_record['INPUT_FIELD'] = {
     22                 i["name"]: InputRecordField(**i)



ipdb>  n


> /tmp/ipykernel_370706/3292963255.py(21)connect_input_record_with_match_record()
     19         import pdb; pdb.set_trace()
     20         for input_record in payload[cn.ALERT_FIELD][cn.INPUT_RECORD_HIST]:
---> 21             input_record['INPUT_FIELD'] = {
     22                 i["name"]: InputRecordField(**i)
     23                 for i in input_record['field']



ipdb>  


> /tmp/ipykernel_370706/3292963255.py(23)connect_input_record_with_match_record()
     21             input_record['INPUT_FIELD'] = {
     22                 i["name"]: InputRecordField(**i)
---> 23                 for i in input_record['field']
     24             }
     25 



ipdb>  


> /tmp/ipykernel_370706/3292963255.py(20)connect_input_record_with_match_record()
     18     def connect_input_record_with_match_record(self, payload):
     19         import pdb; pdb.set_trace()
---> 20         for input_record in payload[cn.ALERT_FIELD][cn.INPUT_RECORD_HIST]:
     21             input_record['INPUT_FIELD'] = {
     22                 i["name"]: InputRecordField(**i)



ipdb>  


> /tmp/ipykernel_370706/3292963255.py(21)connect_input_record_with_match_record()
     19         import pdb; pdb.set_trace()
     20         for input_record in payload[cn.ALERT_FIELD][cn.INPUT_RECORD_HIST]:
---> 21             input_record['INPUT_FIELD'] = {
     22                 i["name"]: InputRecordField(**i)
     23                 for i in input_record['field']



ipdb>  c


# Transform standardized to application

In [20]:
payload = new_payloads

In [21]:
new_payloads = payload
for payload in new_payloads:
    matches = payload[cn.ALERT_FIELD][cn.MATCH_RECORDS]
    agent_config, yaml_conf = self.load_agent_config()
    agent_input_prepended_agent_name_config = prepend_agent_name_to_ap_or_wl_or_aliases_key(
        agent_config
    )

    agent_input_agg_col_config = create_agent_input_agg_col_config(
        agent_input_prepended_agent_name_config
    )

    for match in matches:
        config = self.get_key(payload, match, yaml_conf)
        self.engine.sql_to_merge_specific_columns_to_standardized(
            agent_input_prepended_agent_name_config,
            match,
            config,
            False,
        )
        config.update(
            {
                key: match.get(key)
                for key in match
                if key.endswith("_ap") or key.endswith("_wl")
            }
        )
        self.engine.sql_to_merge_specific_columns_to_standardized(
            agent_input_agg_col_config, match, config, True
        )



### payload

In [22]:
new_payloads[0] is new_payloads[1]

False

In [23]:
for payload in new_payloads:
    print('p')
    for match in payload['alert']['matchRecords']:
        print('match')
        print("++++++++")
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


p
match
++++++++
[['02/31/1900', '04/31/1910']] [['MAY 6, 1981']]
['UNITED STATES OF AMERICA'] ['US']
p
match
++++++++
[['02/31/1900', '04/31/1910']] [['MAY 6, 1981']]
['UNITED STATES OF AMERICA'] ['PL']


In [17]:
import pickle
with open("tests/shared/parsed_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [24]:
new_dict = {}
def return_key(dict_, prefix):
    
    if isinstance(dict_, list):
        for num, i in enumerate(dict_):
            return_key(i, prefix +f"[{num}]")
        return
    if isinstance(dict_, str) or dict_ is None:
        new_dict[prefix] = dict_
        return

    for key in dict_:
        if prefix:
            basic_prefix = prefix + "." + key
        else:
            basic_prefix = key
        return_key(dict_[key], basic_prefix)

In [27]:
return_key(payload, "")

In [ ]:
with open(f'notebooks/sample/alert_in_payload_format.json', 'w') as file:
    json.dump(new_dict ,file)

In [ ]:
payload_json = PayloadLoader().load_payload_from_json(new_dict)

In [ ]:
payload_json == payload

In [26]:
with open(f'notebooks/sample/alert.json', 'r') as file:
    payload = json.loads(file.read())

In [ ]:
payload